<a href="https://colab.research.google.com/github/tfreyd/TensorflowCoursera/blob/main/C4/W4/ungraded_labs/C4_W4_Lab_1_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.6.0


In [11]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [12]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [13]:
def model_forecast(model, series, window_size):
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size, shift=1, drop_remainder=True)
  ds = ds.falt_map(lambda w: w.batch(window_size))
  ds = ds.batch(32).prefetch(1)
  forecast = model.predict(ds)
  return forecast

In [16]:
tf.keras.backend.clear_session() 
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30 
train_set = windowed_dataset ( x_train, window_size, batch_size=128, shuffle_buffer = shuffle_buffer_size)

model = tf.keras.models.Sequential ([
  tf.keras.layers.Conv1D(32, kernel_size=5,strides=1, padding = 'causal',activation='relu',input_shape=[None,1]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x : x * 200)                     
])


#setting up the callback to find the best learning rate 
lr_schedule = tf.keras.callbacks.LearningRateScheduler( lambda epoch: 1e-8 * 10**(epoch/20))

optimizer  = tf.keras.optimizers.SGD(learning_rate=1e-8, momentum= 0.9)

model.compile(loss=tf.keras.losses.Huber(),
optimizer = optimizer, metrics=['mae'])

history= model.fit(train_set, epochs = 100, callbacks = [lr_schedule])

Epoch 1/100
8/8 [==============================] - 5s 13ms/step - loss: 34.0438 - mae: 34.5416
Epoch 2/100
8/8 [==============================] - 0s 12ms/step - loss: 33.0697 - mae: 33.5672
Epoch 3/100
8/8 [==============================] - 0s 13ms/step - loss: 31.4026 - mae: 31.8998
Epoch 4/100
8/8 [==============================] - 0s 12ms/step - loss: 29.2024 - mae: 29.6995
Epoch 5/100
8/8 [==============================] - 0s 13ms/step - loss: 26.4395 - mae: 26.9352
Epoch 6/100
8/8 [==============================] - 0s 12ms/step - loss: 23.2858 - mae: 23.7805
Epoch 7/100
8/8 [==============================] - 0s 13ms/step - loss: 20.0954 - mae: 20.5887
Epoch 8/100
8/8 [==============================] - 0s 12ms/step - loss: 17.4126 - mae: 17.9056
Epoch 9/100
8/8 [==============================] - 0s 11ms/step - loss: 15.5792 - mae: 16.0712
Epoch 10/100
8/8 [==============================] - 0s 12ms/step - loss: 14.5424 - mae: 15.0343
Epoch 11/100
8/8 [==============================]